In [2]:
import datetime as dt
import pandas as pd
import requests
import folium
import json

#print('importado')

In [3]:
boundaries_map = r'res/custom.geo.json'

In [4]:
#FUNDOS DE PREVIDÊNCIA COMO PORCENTAGEM DO PIB
df_share_gdp = pd.read_csv('res/2.1. Assets as share of GDP.csv')

#Eliminando colunas inúteis
df_share_gdp.drop('Pension Plan Type', axis=1, inplace=True)
df_share_gdp.drop('PPTYP', axis=1, inplace=True)
df_share_gdp.drop('DTYP', axis=1, inplace=True)
df_share_gdp.drop('Definition Type', axis=1, inplace=True)
df_share_gdp.drop('CTYP', axis=1, inplace=True)
df_share_gdp.drop('Contract Type', axis=1, inplace=True)
df_share_gdp.drop('VAR', axis=1, inplace=True)
df_share_gdp.drop('Variable', axis=1, inplace=True)
df_share_gdp.drop('IND', axis=1, inplace=True)
df_share_gdp.drop('Indicator', axis=1, inplace=True)
df_share_gdp.drop('Flags', axis=1, inplace=True)
df_share_gdp.drop('Flag Codes', axis=1, inplace=True)
df_share_gdp.drop('Reference Period', axis=1, inplace=True)
df_share_gdp.drop('Reference Period Code', axis=1, inplace=True)
df_share_gdp.drop('PowerCode', axis=1, inplace=True)
df_share_gdp.drop('PowerCode Code', axis=1, inplace=True)
df_share_gdp.drop('Unit', axis=1, inplace=True)
df_share_gdp.drop('YEA', axis=1, inplace=True)

#Dividindo dataframes por anos
df_share_gdp_2008=df_share_gdp[df_share_gdp['Year'] == 2008]
df_share_gdp_2008.reset_index(inplace=True, drop=True)
df_share_gdp_2009=df_share_gdp[df_share_gdp['Year'] == 2009]
df_share_gdp_2009.reset_index(inplace=True, drop=True)
df_share_gdp_2010=df_share_gdp[df_share_gdp['Year'] == 2010]
df_share_gdp_2010.reset_index(inplace=True, drop=True)
df_share_gdp_2011=df_share_gdp[df_share_gdp['Year'] == 2011]
df_share_gdp_2011.reset_index(inplace=True, drop=True)
df_share_gdp_2012=df_share_gdp[df_share_gdp['Year'] == 2012]
df_share_gdp_2012.reset_index(inplace=True, drop=True)
df_share_gdp_2013=df_share_gdp[df_share_gdp['Year'] == 2013]
df_share_gdp_2013.reset_index(inplace=True, drop=True)
df_share_gdp_2014=df_share_gdp[df_share_gdp['Year'] == 2014]
df_share_gdp_2014.reset_index(inplace=True, drop=True)
df_share_gdp_2015=df_share_gdp[df_share_gdp['Year'] == 2015]
df_share_gdp_2015.reset_index(inplace=True, drop=True)
df_share_gdp_2016=df_share_gdp[df_share_gdp['Year'] == 2016]
df_share_gdp_2016.reset_index(inplace=True, drop=True)
df_share_gdp_2017=df_share_gdp[df_share_gdp['Year'] == 2017]
df_share_gdp_2017.reset_index(inplace=True, drop=True)
df_share_gdp_2018=df_share_gdp[df_share_gdp['Year'] == 2018]
df_share_gdp_2018.reset_index(inplace=True, drop=True)

df_share_gdp_2018.head(10)

,COU,Country,Year,Unit Code,Value
0,AUS,Australia,2018,PC,137.864
1,AUT,Austria,2018,PC,5.550
2,BEL,Belgium,2018,PC,6.826
3,CAN,Canada,2018,PC,84.390
4,CZE,Czech Republic,2018,PC,9.171
5,DNK,Denmark,2018,PC,45.271
6,FIN,Finland,2018,PC,49.740
7,FRA,France,2018,PC,0.705
8,DEU,Germany,2018,PC,6.901
9,GRC,Greece,2018,PC,0.749


In [15]:
#GASTOS COM PREVIDÊNCIA COM PORCENTAGEM DO PIB
df_gdp_spending = pd.read_csv('res/1.1. OECD Pensions as share GDP.csv', header=0)

#Eliminando colunas inúteis
df_gdp_spending.drop('INDICATOR', axis=1, inplace=True)
df_gdp_spending.drop('MEASURE', axis=1, inplace=True)
df_gdp_spending.drop('FREQUENCY', axis=1, inplace=True)
df_gdp_spending.drop('Flag Codes', axis=1, inplace=True)

#Dropa linhas de gasto privado
i = 0
for gasto in df_gdp_spending['SUBJECT']:
    if gasto == 'PRIV':
        df_gdp_spending.drop([i], axis=0, inplace=True)

    i+=1
df_gdp_spending.reset_index(inplace=True,drop=True)

df_gdp_spending.head(10)

,LOCATION,SUBJECT,TIME,Value
0,AUS,PUB,1980,3.593
1,AUS,PUB,1981,3.555
2,AUS,PUB,1982,3.675
3,AUS,PUB,1983,3.627
4,AUS,PUB,1984,3.560
5,AUS,PUB,1985,3.399
6,AUS,PUB,1986,3.291
7,AUS,PUB,1987,3.239
8,AUS,PUB,1988,2.892
9,AUS,PUB,1989,2.843


In [6]:
#Inicializando mapa
map = folium.Map(location=[30,5], zoom_start=1.9, min_zoom=1.9, max_bounds=True)

folium.GeoJson(
    boundaries_map,
    name='fronteiras',
    style_function=lambda feature: {
        'fillColor': 'red',
        'color' : 'red',
        'weight' : 1,
        'fillOpacity' : 0.5,
        }
).add_to(map)

map.save(outfile='map.html')

map

In [7]:
a = open('res/custom.geo.json')
a = json.load(a)
i=0

c = []

for co in a['features']:
    c.append(a['features'][i]['properties']['admin'])
    print(a['features'][i]['properties']['admin'])
    
    i+=1

The Bahamas
Belize
Canada
Costa Rica
Cuba
Dominican Republic
Guatemala
Honduras
Haiti
Jamaica
Greenland
Mexico
Nicaragua
Panama
El Salvador
Puerto Rico
Trinidad and Tobago
United States of America
Argentina
Bolivia
Chile
Colombia
Ecuador
Falkland Islands
Guyana
Peru
Brazil
Suriname
Uruguay
Venezuela
Afghanistan
United Arab Emirates
Paraguay
Bangladesh
Brunei
Armenia
Azerbaijan
China
Northern Cyprus
Cyprus
Bhutan
Georgia
Indonesia
India
Iran
Iraq
Israel
Jordan
Japan
Kyrgyzstan
Cambodia
South Korea
Kuwait
Kazakhstan
Myanmar
Mongolia
Laos
Lebanon
Sri Lanka
Malaysia
Nepal
Pakistan
Oman
North Korea
Palestine
Qatar
Philippines
Saudi Arabia
Syria
Thailand
Tajikistan
Turkmenistan
East Timor
Taiwan
Turkey
Uzbekistan
Vietnam
Yemen
Angola
Burundi
Benin
Burkina Faso
Botswana
Central African Republic
Ivory Coast
Cameroon
Democratic Republic of the Congo
Republic of Congo
Djibouti
Algeria
Egypt
Eritrea
Ethiopia
Gabon
Gambia
Ghana
Guinea
Guinea Bissau
Equatorial Guinea
Kenya
Liberia
Libya
Lesotho
Mor

In [8]:
#Lista com todos os países do geojson
countr_dict = {'Country': c}
countr_dict['Country']

['The Bahamas',
 'Belize',
 'Canada',
 'Costa Rica',
 'Cuba',
 'Dominican Republic',
 'Guatemala',
 'Honduras',
 'Haiti',
 'Jamaica',
 'Greenland',
 'Mexico',
 'Nicaragua',
 'Panama',
 'El Salvador',
 'Puerto Rico',
 'Trinidad and Tobago',
 'United States of America',
 'Argentina',
 'Bolivia',
 'Chile',
 'Colombia',
 'Ecuador',
 'Falkland Islands',
 'Guyana',
 'Peru',
 'Brazil',
 'Suriname',
 'Uruguay',
 'Venezuela',
 'Afghanistan',
 'United Arab Emirates',
 'Paraguay',
 'Bangladesh',
 'Brunei',
 'Armenia',
 'Azerbaijan',
 'China',
 'Northern Cyprus',
 'Cyprus',
 'Bhutan',
 'Georgia',
 'Indonesia',
 'India',
 'Iran',
 'Iraq',
 'Israel',
 'Jordan',
 'Japan',
 'Kyrgyzstan',
 'Cambodia',
 'South Korea',
 'Kuwait',
 'Kazakhstan',
 'Myanmar',
 'Mongolia',
 'Laos',
 'Lebanon',
 'Sri Lanka',
 'Malaysia',
 'Nepal',
 'Pakistan',
 'Oman',
 'North Korea',
 'Palestine',
 'Qatar',
 'Philippines',
 'Saudi Arabia',
 'Syria',
 'Thailand',
 'Tajikistan',
 'Turkmenistan',
 'East Timor',
 'Taiwan',
 'Tur

In [9]:
df_share_gdp_2018['Country']

0          Australia
1            Austria
2            Belgium
3             Canada
4     Czech Republic
           ...      
72          Malaysia
73            Guyana
74         Singapore
75        Kazakhstan
76        Mozambique
Name: Country, Length: 77, dtype: object

In [17]:
#Mapa coroplético
map2 = folium.Map(location=[30,5], zoom_start=1.8, min_zoom=1.8, max_bounds=True)

folium.Choropleth(
    geo_data=boundaries_map,
    data=df_gdp_spending,
    columns=['LOCATION', 'Value'],
    key_on='feature.properties.adm0_a3',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2
).add_to(map2)

map2

In [16]:
a

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'scalerank': 1,
    'featurecla': 'Admin-0 country',
    'labelrank': 4,
    'sovereignt': 'The Bahamas',
    'sov_a3': 'BHS',
    'adm0_dif': 0,
    'level': 2,
    'type': 'Sovereign country',
    'admin': 'The Bahamas',
    'adm0_a3': 'BHS',
    'geou_dif': 0,
    'geounit': 'The Bahamas',
    'gu_a3': 'BHS',
    'su_dif': 0,
    'subunit': 'The Bahamas',
    'su_a3': 'BHS',
    'brk_diff': 0,
    'name': 'Bahamas',
    'name_long': 'Bahamas',
    'brk_a3': 'BHS',
    'brk_name': 'Bahamas',
    'brk_group': None,
    'abbrev': 'Bhs.',
    'postal': 'BS',
    'formal_en': 'Commonwealth of the Bahamas',
    'formal_fr': None,
    'note_adm0': None,
    'note_brk': None,
    'name_sort': 'Bahamas, The',
    'name_alt': None,
    'mapcolor7': 1,
    'mapcolor8': 1,
    'mapcolor9': 2,
    'mapcolor13': 5,
    'pop_est': 309156,
    'gdp_md_est': 9093,
    'pop_year': -99,
    'lastcensus': 2010,
    'gdp_y